In [21]:
import numpy as np
import pandas as pd
import pytz
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

instantiate date/time objects

In [6]:
x = pd.to_datetime("2024-05-06")
x = pd.to_datetime("2024-05-06 23:00:00+08:00")
x = pd.to_datetime("2024-05-06 23:00:00").tz_localize("UTC")
x.tz_localize("America/Chicago")
x.tz_convert("Asia/Hong_Kong")
x.value # timestampe value in ns

# datetime index
dates = pd.date_range('2024-01-01', '2024-12-31', freq="WOM-3FRI")
dates = pd.date_range('2024-01-01', '2024-12-31', freq="B")

In [30]:
df = pd.DataFrame(index=dates, data=dates)

In [ ]:
df.resample('1M').agg('first')

shift time/ date

In [ ]:
x + timedelta()
x + relativedelta(months=3)

example in shifting data

In [43]:
dates = pd.date_range('2024-01-01', '2024-12-31', freq='B')
data = np.random.standard_normal(len(dates))
df = pd.DataFrame(index=dates, data=data, columns=['original'])

In [60]:
df['rolling_mean'] = df['original'].rolling(5).mean()
df['pct_change'] = df['original'].pct_change()

In [61]:
df

,original,rolling_mean,pct_change
2024-01-01,0.149956,NaN,NaN
2024-01-02,-0.420670,NaN,-3.805300
2024-01-03,1.552616,NaN,-4.690815
2024-01-04,0.292398,NaN,-0.811674
2024-01-05,-1.857487,-0.056637,-7.352590
...,...,...,...
2024-12-25,-2.195142,-0.573381,-15.565474
2024-12-26,0.606111,-0.277179,-1.276115
2024-12-27,0.067823,-0.163669,-0.888101
2024-12-30,1.046488,-0.064802,14.429706
